<a href="https://colab.research.google.com/github/AnnaKraim/ML/blob/main/dota2_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Импортируем нужные для работы библиотеки
import pandas as pd
import numpy as np
import datetime
import warnings

warnings.filterwarnings("ignore")

# Считываем данные
df = pd.read_csv('features_small.csv', index_col='match_id')
df.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
49349,1444882995,0,11,4,1468,1439,15,1,1,5,...,5,3,1,-22.0,2412,0,0.0,2047.0,0.0,63.0
22565,1438857010,7,3,2,545,792,0,2,1,16,...,2,2,0,-26.0,3224,1,1980.0,0.0,63.0,0.0
66647,1447236848,1,9,3,865,588,2,0,0,9,...,4,2,1,-66.0,2619,0,1536.0,1846.0,0.0,63.0
75024,1448212736,7,47,4,1130,1202,8,1,1,8,...,4,2,0,272.0,2248,0,256.0,2047.0,48.0,63.0
82416,1448668264,1,20,2,405,661,1,0,0,9,...,4,3,0,20.0,3351,1,1796.0,0.0,55.0,0.0


Это датасет по **Dota 2**

Предсказание исход игры по первым минутам матча (первые 5)

## Предобработка данных

Удалим признаки, содержащие ответ

In [ ]:
y = df['radiant_win']
X = df.drop(columns=['start_time', 'radiant_win', 'duration', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'])
X.head()

,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
49349,0,11,4,1468,1439,15,1,1,5,16,...,0,NaN,79.0,-85.0,226.0,1,5,3,1,-22.0
22565,7,3,2,545,792,0,2,1,16,62,...,1,19.0,175.0,-84.0,300.0,5,2,2,0,-26.0
66647,1,9,3,865,588,2,0,0,9,37,...,0,33.0,129.0,-83.0,NaN,4,4,2,1,-66.0
75024,7,47,4,1130,1202,8,1,1,8,39,...,0,55.0,207.0,-82.0,NaN,3,4,2,0,272.0
82416,1,20,2,405,661,1,0,0,9,67,...,2,18.0,62.0,-76.0,271.0,1,4,3,0,20.0


Удалим пропуски в данных

In [ ]:
X = X.fillna(0)
X.head()

,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
49349,0,11,4,1468,1439,15,1,1,5,16,...,0,0.0,79.0,-85.0,226.0,1,5,3,1,-22.0
22565,7,3,2,545,792,0,2,1,16,62,...,1,19.0,175.0,-84.0,300.0,5,2,2,0,-26.0
66647,1,9,3,865,588,2,0,0,9,37,...,0,33.0,129.0,-83.0,0.0,4,4,2,1,-66.0
75024,7,47,4,1130,1202,8,1,1,8,39,...,0,55.0,207.0,-82.0,0.0,3,4,2,0,272.0
82416,1,20,2,405,661,1,0,0,9,67,...,2,18.0,62.0,-76.0,271.0,1,4,3,0,20.0


## Логистическая регрессия

* Оценим качество логистической регрессии (sklearn.linear_model.LogisticRegression), разбив выборку на train/test. 

Качество в обоих случаях

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred_train=lr.predict(X_train)
y_pred_test=lr.predict(X_test)
accuracy_score(y_pred_train,y_train),accuracy_score(y_pred_test,y_test)

(0.6570552477271505, 0.6515945827872434)

In [ ]:
from sklearn.linear_model import Ridge
clf = Ridge(2.0)
cross_val_score(clf, X, y, cv = 5).mean()

0.13905198914048028

In [ ]:
cross_val_score(LogisticRegression(C = 2), X, y, cv= 5, scoring = 'f1').mean()

0.6670630098946061

* Оценим качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по 5 блокам (без разбиения на train/test). (cross-val-score)

Наилучший параметр C и качество

(Цикл / grid_search_cv)

In [ ]:
from sklearn.model_selection import GridSearchCV
logr = LogisticRegression()

params = {'C': np.arange(0, 3,0.1)}

logr = GridSearchCV(logr, params, verbose=1, n_jobs=-1)
logr.fit(X, y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
       2.6, 2.7, 2.8, 2.9])},
             verbose=1)

In [ ]:
logr.best_score_, logr.best_params_

(0.653631284916201, {'C': 2.1})

Отмасштабируем признаки с помощью **MinMaxScaler**

Проверка на кросс-валидации на 5 блоках

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(X)

X_scaled = scaler.transform(X)

cross_val_score(LogisticRegression(C = 1.1), X_scaled, y, cv= 5, scoring = 'f1').mean()

0.6699137620695049

## Категориальные признаки

Создадим список с названиями всех категориальных признаков.

In [ ]:

categorial = ["lobby_type", ]
for i in range(1, 6):
  categorial.append("r{}_hero".format(i))
  categorial.append("d{}_hero".format(i))
categorial

['lobby_type',
 'r1_hero',
 'd1_hero',
 'r2_hero',
 'd2_hero',
 'r3_hero',
 'd3_hero',
 'r4_hero',
 'd4_hero',
 'r5_hero',
 'd5_hero']

#### Кол-во различных героев в игре

In [ ]:
N = 112

Столбцы с категориальными признаками

In [ ]:
X_nocat = X.drop(columns=categorial)
X_nocat.head()


,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,r2_gold,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
49349,4,1468,1439,15,1,1,5,3,1058,782,...,0,0.0,79.0,-85.0,226.0,1,5,3,1,-22.0
22565,2,545,792,0,2,1,16,3,1101,1284,...,1,19.0,175.0,-84.0,300.0,5,2,2,0,-26.0
66647,3,865,588,2,0,0,9,2,714,574,...,0,33.0,129.0,-83.0,0.0,4,4,2,1,-66.0
75024,4,1130,1202,8,1,1,8,4,1472,1060,...,0,55.0,207.0,-82.0,0.0,3,4,2,0,272.0
82416,2,405,661,1,0,0,9,4,1459,1307,...,2,18.0,62.0,-76.0,271.0,1,4,3,0,20.0


Проверка качества модели без категориальных переменных

In [ ]:
cross_val_score(LogisticRegression(C = 1.1), X_nocat, y, cv= 5, scoring = 'f1').mean()

0.6687073514278241

## Кодирование категориальных признаков

OneHotEncoder — метод, который кодирует категориальные признаки с помощью вектора (0, 1).


В данной задаче применять OneHotEncoder не стоит.

У нас есть около 110 героев. Есть 10 категориальных признаков, которые содержат ~110 героев. Получаем 
10 * 110 = 1100 новых признаков.


Если подумать, то нам **не важно**(по крайней мере в данной задаче) **какой** игрок играл на каком герое. Достаточно знать, что герой был либо на стороне radiant, либо на стороне dire.

Тогда мы можем сделать всего-лишь столько признаков, сколько героев в игре. И ставить **+1** в признак, если герой был у radiant и **-1**, если у dire.

In [ ]:
# Сформируем табличку с героями

n_heroes = 112# получите это число сами
X_pick = np.zeros((X.shape[0], n_heroes))

for i, match_id in enumerate(X.index):
    for p in range(5):
        X_pick[i, X.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, X.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1


    

In [ ]:
X_pick = pd.DataFrame(X_pick, index = X.index, columns=["hero_{}".format(x) for x in range(n_heroes)])
X_pick.head()

,hero_0,hero_1,hero_2,hero_3,hero_4,hero_5,hero_6,hero_7,hero_8,hero_9,...,hero_102,hero_103,hero_104,hero_105,hero_106,hero_107,hero_108,hero_109,hero_110,hero_111
match_id,,,,,,,,,,,,,,,,,,,,,
49349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
22565,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66647,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75024,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Теперь, используя pd.Concat, создадим **новый** DataFrame, который учитывает наших героев.


In [ ]:
cotic = pd.concat([X_nocat,X_pick], axis=1)
cotic.head


<bound method NDFrame.head of           r1_level  r1_xp  r1_gold  r1_lh  r1_kills  r1_deaths  r1_items  \
match_id                                                                   
49349            4   1468     1439     15         1          1         5   
22565            2    545      792      0         2          1        16   
66647            3    865      588      2         0          0         9   
75024            4   1130     1202      8         1          1         8   
82416            2    405      661      1         0          0         9   
...            ...    ...      ...    ...       ...        ...       ...   
42923            3   1031     1651     27         0          0        12   
33544            4   1602     1553     20         1          0        10   
26109            5   2182     1505     17         1          0         7   
10142            5   2131      988     12         0          0         7   
92693            3    833      765      8         0       

Ну и проверим, улучшилось ли качество. 

Кросс-валидация 5 блоков, все дела

In [ ]:
cross_val_score(LogisticRegression(C = 1.1), cotic, y, cv= 5, scoring = 'f1').mean()

0.6680828990637022

Далее мы будем использовать F-меру в качестве метрики к данной задаче. 

Оценка метрики, используя кросс-валидацию по 5 блоками.

Подбор параметров для логистической регрессии, используя кросс-валидацию для наилучшего результата.


# **↑**

### Случайный лес и градиентный бустинг. 

Используя алгоритм случайного леса или градиентного бустинга попробуем получить качество лучше, чем показала логистическая регрессия. Подберем параметры и посмотрим качество.

In [ ]:
from sklearn.metrics import make_scorer
mse_scoring = make_scorer(make_scorer)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
params = {
    'n_estimators': range(50,150),
    'subsample': np.arange(0.1,1,0.1)
}

gcv = GridSearchCV(GradientBoostingClassifier(),params,verbose = 1, n_jobs = -1, scoring = mse_scoring)

gcv.fit(X,y)

Fitting 5 folds for each of 900 candidates, totalling 4500 fits
